# 

In [1]:
import dora
import pyarrow as pa
import inspect

In [2]:
node = dora.Node()

In [43]:
import os
import sys


## Get all python files path in given directory
def get_all_functions(path):
    functions = []
    paths = []
    codes = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".py"):
                path = os.path.join(root, file)
                with open(path, "r", encoding="utf8") as f:
                    ## add file folder to system path
                    sys.path.append(root)
                    ## import module from path
                    try:
                        module = __import__(file[:-3])
                        ## get all functions in module
                        for name, data in module.__dict__.items():
                            # if inspect.isfunction(data):
                            # ## Get function source code
                            # print(data)
                            # source = inspect.getsource(data)
                            # functions.append(source)
                            # paths.append(path)
                            if inspect.isclass(data) and name == "Operator":
                                # ## Get function source code
                                # print(data)
                                source = inspect.getsource(data.on_input)
                                codes.append(source)
                                functions.append(f.read())
                                paths.append(file)

                    except Exception as err:
                        print(err)
                        pass

    return functions, paths, codes


path = "/home/peter/Documents/work/dora-drives"
files, paths, codes = get_all_functions(path)
print(f"Number of operators to watch {len(files)}")

/home/peter/miniconda3/lib/python3.10/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /lib/x86_64-linux-gnu/libicuuc.so.72)
No module named 'open3d'
No module named 'yolov7_tt100k'
No module named 'carla'
No module named 'autoagents'
No module named 'carla'
No module named 'carla'
No module named 'carla'
No module named 'carla'
No module named 'rospy'
No module named 'mavsdk'
No module named 'rospy'
No module named 'sensor_msgs'
No module named 'sensor_msgs'
No module named 'sensor_msgs'
Number of operators to watch 9


In [4]:
files.extend(paths)
files.extend(codes)
node.send_output("append", pa.array(files))

In [5]:
node.send_output("query", pa.array(["Can you only show 10 bounding boxes?"]))

In [6]:
event = node.next(timeout=10)

In [7]:
data = event["value"]

In [8]:
data.to_pandas()

0                                          yolov5_op.py
1     """ \n# Yolov5 operator\n\n`Yolov5` object det...
2         def on_input(\n        self,\n        dora...
3                                     strong_sort_op.py
4     """ \n# Strong Sort operator\n\n`Strong sort` ...
5         def on_input(\n        self,\n        dora...
6                               obstacle_location_op.py
7     """ \n# Obstacle location operator\n\nThe obst...
8         def on_input(\n        self,\n        dora...
9                                               plot.py
10    import time\nfrom typing import Callable\n\nim...
11        def on_input(\n        self,\n        dora...
12                                          yolop_op.py
13    """ \n# yolop operator\n\n`yolop` recognizes l...
14        def on_input(\n        self,\n        dora...
dtype: object

In [16]:
text = data[2]

In [10]:
node.send_output("prompt", pa.array([f"print hello world?"]))

In [11]:
event = node.next(timeout=10)

In [12]:
event["value"]

[
  "        
        To print "hello world" in various programming languages, you can use the following code snippets:

        Python:
        ```python
        print("Hello World!")
        ```
        
        JavaScript:
        ```javascript
        console.log("Hello World!");
        ```
        
        Java:
        ```java
        public class HelloWorld {
            public static void main(String[] args) {
                System.out.println("Hello World!");
            }
        }
        ```
        
        C:
        ```c
        #include <stdio.h>
        int main() {
            printf("Hello World!\n");
            return 0;
        }
        ```
        
        C++:
        ```c++
        #include <iostream>
        int main() {
            std::cout << "Hello World!" << std::endl;
            return 0;
        }
        ```
        
        Pick the language you're using and execute the corresponding code snippet to print "Hello World"."
]

In [39]:
node.send_output("prompt", pa.array([f"{text}  \n Can you filter to show 10 bounding boxes. Respond with code only"]))

In [40]:
event = node.next(timeout=10)

In [41]:
event["value"]

[
  "        Sure, here's how you can modify the code to display the top 10 bounding boxes:

```python
def on_input(self, dora_input: dict, send_output: Callable[[str, bytes], None]):
    """
    Handle image
    Args:
        dora_input["id"] (str): Id of the input declared in the yaml configuration
        dora_input["value"] (arrow.array (UInt8)): Bytes message of the input
        send_output (Callable[[str, bytes]]): Function enabling sending output back to dora.
    """
    if dora_input["id"] == "image":
        frame = dora_input["value"].to_numpy().reshape((IMAGE_HEIGHT, IMAGE_WIDTH, 4))
        frame = frame[:, :, :3]

        results = self.model(frame)  # includes NMS

        # Sort the results by confidence score (highest first)
        results.xyxy = sorted(results.xyxy, key=lambda x: x[5], reverse=True)

        # Limit the output to the top 10 bounding boxes
        results = results[:10]

        arrays = [np.array(r.xyxy[0].cpu()).reshape(-1) for r in results]
      

In [ ]:
node.send_output("clear", pa.array([""]))